In [1]:
import pandas as pd
import numpy as np
from scipy.stats import spearmanr, pearsonr
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler

In [2]:
### 시각화를 위한 라이브러리
# - 파이썬 시각화에서 가장 기본적인 시각화 라이브러리
import matplotlib.pyplot as plt

### 한글처리
from matplotlib import font_manager, rc

### 운영체제 확인 라이브러리
import platform

### 시각화 시 마이너스(-, 음수) 기호 깨짐 방지
plt.rcParams["axes.unicode_minus"] = False

### OS별 한글처리
# - 윈도우 운영체게
if platform.system() == "Windows" :
    # path = "c:/Windows/Fonts/malgun.ttf"
    # font_name = font_manager.FontProperties(fname=path).get_name()
    # rc("font", family = font_name)
    
    ### 또는 아래처럼 한줄로도 가능 (아래 한글처리를 주로 사용합니다.)
    plt.rc("font", family = "Malgun Gothic")

# - Mac 운영체제
elif platform.system() == "Darwin" :
    rc("font", family = "AppleGothic")
    
else :
    print("넌 누구?")

In [3]:
labeled = pd.read_csv("./../25day/data/labeled_data.csv")

In [4]:
labeled_cn7 = labeled[(labeled["PART_NAME"] == "CN7 W/S SIDE MLD'G RH") |(labeled["PART_NAME"] == "CN7 W/S SIDE MLD'G LH")]

In [5]:
labeled_cn7 = labeled_cn7.drop(columns=["_id", "TimeStamp","PART_FACT_PLAN_DATE", "Reason", "PART_FACT_SERIAL", "PART_NAME",
               "EQUIP_NAME", "EQUIP_CD", "Switch_Over_Position", "Mold_Temperature_1","Mold_Temperature_2","Mold_Temperature_5",
                "Mold_Temperature_6", "Mold_Temperature_7","Mold_Temperature_8","Mold_Temperature_9","Mold_Temperature_10",
               "Mold_Temperature_11","Mold_Temperature_12","Barrel_Temperature_7"])

In [6]:
labeled_cn7

,PassOrFail,Injection_Time,Filling_Time,Plasticizing_Time,Cycle_Time,Clamp_Close_Time,Cushion_Position,Plasticizing_Position,Clamp_Open_Position,Max_Injection_Speed,...,Average_Back_Pressure,Barrel_Temperature_1,Barrel_Temperature_2,Barrel_Temperature_3,Barrel_Temperature_4,Barrel_Temperature_5,Barrel_Temperature_6,Hopper_Temperature,Mold_Temperature_3,Mold_Temperature_4
0,Y,9.59,4.47,16.920000,59.520000,7.13,653.409973,68.849998,647.98999,55.400002,...,59.299999,276.500000,274.700012,274.799988,269.200012,255.000000,229.699997,66.300003,24.799999,27.500000
1,Y,9.60,4.48,16.910000,59.580002,7.13,653.409973,68.839996,647.98999,55.299999,...,59.299999,276.200012,275.500000,275.299988,270.799988,254.699997,229.500000,67.199997,24.799999,27.600000
2,Y,9.60,4.48,16.910000,59.580002,7.13,653.409973,68.839996,647.98999,55.299999,...,59.299999,276.200012,275.500000,275.299988,270.799988,254.699997,229.500000,67.199997,24.799999,27.600000
3,Y,9.59,4.48,16.910000,59.560001,7.13,653.419983,68.839996,647.98999,55.299999,...,59.299999,276.500000,275.000000,275.399994,271.100006,254.899994,230.000000,66.900002,25.000000,27.600000
4,Y,9.59,4.48,16.910000,59.560001,7.13,653.419983,68.839996,647.98999,55.299999,...,59.299999,276.500000,275.000000,275.399994,271.100006,254.899994,230.000000,66.900002,25.000000,27.600000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7991,Y,9.60,4.48,16.620001,59.560001,7.11,653.429993,68.330002,647.98999,54.900002,...,59.900002,276.500000,274.899994,275.100006,269.500000,255.600006,229.600006,66.099998,21.000000,22.400000
7992,Y,9.60,4.48,16.650000,59.560001,7.11,653.429993,68.349998,647.98999,55.000000,...,59.900002,275.799988,275.299988,275.500000,270.700012,254.899994,230.000000,66.099998,21.000000,22.400000
7993,Y,9.60,4.48,16.650000,59.560001,7.11,653.429993,68.349998,647.98999,55.000000,...,59.900002,275.799988,275.299988,275.500000,270.700012,254.899994,230.000000,66.099998,21.000000,22.400000
7994,Y,9.60,4.48,16.629999,59.580002,7.11,653.429993,68.330002,647.98999,54.900002,...,59.900002,276.100006,275.799988,275.000000,271.299988,255.100006,230.199997,65.199997,21.000000,22.299999


In [7]:
labeled_cn7["PassOrFail"] = labeled_cn7["PassOrFail"].replace('Y',1).replace('N',0)

C:\Users\Admin\AppData\Local\Temp\ipykernel_10472\3829480872.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  labeled_cn7["PassOrFail"] = labeled_cn7["PassOrFail"].replace('Y',1).replace('N',0)


In [8]:
### Clamp_Open_Position에서 눈에띄게 낮은 값들 제거
cn7 = labeled_cn7[labeled_cn7["Clamp_Open_Position"]>=200] 

In [9]:
cn7 = cn7.drop(columns=["Clamp_Open_Position"])

In [10]:
def remove_outliers(df, column):
    # 1사분위수(Q1)와 3사분위수(Q3) 계산
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    
    # IQR 계산
    IQR = Q3 - Q1
    
    # 이상치 범위 설정
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # 이상치 제거
    filtered_df = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
    
    return filtered_df

In [11]:
cn7_Y = cn7[cn7["PassOrFail"]==1].copy()
print("CN7의 양품 개수:", len(cn7_Y))
cn7_N = cn7[cn7["PassOrFail"]==0].copy()
print("CN7의 불량 개수:", len(cn7_N))

CN7의 양품 개수: 6661
CN7의 불량 개수: 39


In [12]:
tmp1 = cn7_Y.copy()
tmp1 = remove_outliers(tmp1, "Injection_Time")
tmp1 = remove_outliers(tmp1, "Plasticizing_Time")
tmp1 = remove_outliers(tmp1, "Cycle_Time")
tmp1 = remove_outliers(tmp1, "Max_Injection_Speed")
tmp1 = remove_outliers(tmp1, "Mold_Temperature_3")
cn7_pass_removed = tmp1.copy()
cn7_pass_removed.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6534 entries, 0 to 7995
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   PassOrFail                6534 non-null   int64  
 1   Injection_Time            6534 non-null   float64
 2   Filling_Time              6534 non-null   float64
 3   Plasticizing_Time         6534 non-null   float64
 4   Cycle_Time                6534 non-null   float64
 5   Clamp_Close_Time          6534 non-null   float64
 6   Cushion_Position          6534 non-null   float64
 7   Plasticizing_Position     6534 non-null   float64
 8   Max_Injection_Speed       6534 non-null   float64
 9   Max_Screw_RPM             6534 non-null   float64
 10  Average_Screw_RPM         6534 non-null   float64
 11  Max_Injection_Pressure    6534 non-null   float64
 12  Max_Switch_Over_Pressure  6534 non-null   float64
 13  Max_Back_Pressure         6534 non-null   float64
 14  Average_Back_

In [13]:
cn7_removed = pd.concat([cn7_pass_removed, cn7_N], ignore_index=True)
cn7_removed["PassOrFail"].value_counts()

PassOrFail
1    6534
0      39
Name: count, dtype: int64

In [14]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 독립변수와 종속변수 분리 (cn7_removed는 미리 정의된 DataFrame이어야 함)
y = cn7_removed["PassOrFail"]
X = cn7_removed.drop(columns=["PassOrFail"])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=43)

# 데이터 정규화: 학습 데이터에 fit한 후 양쪽을 transform
ss = StandardScaler()
ss.fit(X_train)
X_train_scaled = ss.transform(X_train)
X_test_scaled  = ss.transform(X_test)


In [15]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam
# --- 오토인코더 모델 정의 ---
input_dim = X_train_scaled.shape[1]  # 특성 차원
encoding_dim = 14  # 압축할 차원 수 (원하는 값으로 조정 가능)

# 입력층 정의
input_layer = Input(shape=(input_dim,))
# 인코딩 층 (은닉층)
encoded = Dense(encoding_dim, activation='relu')(input_layer)
# 디코딩 층 (출력층)
decoded = Dense(input_dim, activation='sigmoid')(encoded)

# 오토인코더 모델 생성 및 컴파일
autoencoder = Model(inputs=input_layer, outputs=decoded)
autoencoder.compile(optimizer=Adam(), loss='mean_squared_error')

# --- 오토인코더 모델 학습 ---
autoencoder.fit(X_train_scaled, X_train_scaled,
                epochs=50,
                batch_size=32,
                validation_split=0.2)

# --- 재구성 오차 계산을 통한 이상치 탐지 ---
# 테스트 데이터에 대한 재구성
X_test_pred = autoencoder.predict(X_test_scaled)
# 각 샘플별 재구성 오차(평균 제곱 오차) 계산
reconstruction_error = np.mean(np.square(X_test_scaled - X_test_pred), axis=1)

# 95% 분위수를 기준으로 이상치 판별 임계값 산출
threshold = np.percentile(reconstruction_error, 98)
print("이상치 판별 기준 (재구성 오차):", threshold)

# 재구성 오차가 임계값보다 큰 경우 이상치(불량)로 판단: 0 (불량), 그렇지 않으면 1 (정상)
y_pred = np.where(reconstruction_error > threshold, 0, 1)

# --- 평가 지표 출력 ---
print("Classification Report:")
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])
print("혼돈 행렬:")
print(cm_df)

Epoch 1/50
115/115 [==============================] - 1s 3ms/step - loss: 1.0394 - val_loss: 1.5609
Epoch 2/50
115/115 [==============================] - 0s 993us/step - loss: 0.8607 - val_loss: 1.4158
Epoch 3/50
115/115 [==============================] - 0s 1ms/step - loss: 0.7550 - val_loss: 1.3377
Epoch 4/50
115/115 [==============================] - 0s 1ms/step - loss: 0.7050 - val_loss: 1.2995
Epoch 5/50
115/115 [==============================] - 0s 1ms/step - loss: 0.6790 - val_loss: 1.2755
Epoch 6/50
115/115 [==============================] - 0s 1ms/step - loss: 0.6619 - val_loss: 1.2564
Epoch 7/50
115/115 [==============================] - 0s 1ms/step - loss: 0.6486 - val_loss: 1.2388
Epoch 8/50
115/115 [==============================] - 0s 1ms/step - loss: 0.6375 - val_loss: 1.2232
Epoch 9/50
115/115 [==============================] - 0s 1ms/step - loss: 0.6286 - val_loss: 1.2105
Epoch 10/50
115/115 [==============================] - 0s 1ms/step - loss: 0.6199 - val_loss: 1.20

NameError: name 'classification_report' is not defined

In [17]:
from sklearn.metrics import confusion_matrix, classification_report

threshold = np.percentile(reconstruction_error, 99.6)
print("이상치 판별 기준 (재구성 오차):", threshold)

# 재구성 오차가 임계값보다 큰 경우 이상치(불량)로 판단: 0 (불량), 그렇지 않으면 1 (정상)
y_pred = np.where(reconstruction_error > threshold, 0, 1)

# --- 평가 지표 출력 ---
print("Classification Report:")
print(classification_report(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])
print("혼돈 행렬:")
print(cm_df)

이상치 판별 기준 (재구성 오차): 1.8917642187981338
Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.73      0.84        11
           1       1.00      1.00      1.00      1961

    accuracy                           1.00      1972
   macro avg       1.00      0.86      0.92      1972
weighted avg       1.00      1.00      1.00      1972

혼돈 행렬:
          Predicted 0  Predicted 1
Actual 0            8            3
Actual 1            0         1961
